In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! apt-get install git
! git clone https://github.com/joaorossi15/requirements-to-us.git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'requirements-to-us'...
remote: Enumerating objects: 16002, done.
remote: Counting objects: 100% (5103/5103), done.
remote: Compressing objects: 100% (3756/3756), done.
remote: Total 16002 (delta 1354), reused 5063 (delta 1327), pack-reused 10899 (from 1)
Receiving objects: 100% (16002/16002), 200.81 MiB | 34.47 MiB/s, done.
Resolving deltas: 100% (3502/3502), done.


In [ ]:
import pandas as pd
import numpy as np
from openai import OpenAI

In [ ]:
df = pd.read_csv('./requirements-to-us/data/data.csv')
df = df[['text', 'transparency_sum', 'dignity_sum', 'sustainability_sum', 'trust_sum','freedom_autonomy_sum', 'beneficence_sum', 'privacy_sum', 'responsibility_sum', 'non_maleficence_sum', 'justice_equity_sum']]
df = df.rename(columns={'transparency_sum': 'transparency', 'dignity_sum': 'dignity', 'sustainability_sum': 'sustainability', 'trust_sum': 'trust', 'freedom_autonomy_sum': 'freedom and autonomy', 'beneficence_sum': 'beneficence', 'privacy_sum': 'privacy', 'responsibility_sum': 'responsibility', 'non_maleficence_sum': 'non-maleficence', 'justice_equity_sum': 'justice and equity'})
df.columns

Index(['text', 'transparency', 'dignity', 'sustainability', 'trust',
       'freedom and autonomy', 'beneficence', 'privacy', 'responsibility',
       'non-maleficence', 'justice and equity'],
      dtype='object')

In [ ]:
for i, row in df.iterrows():
    row = row[1:]
    if row.sum() == 0.0:
        df.at[i,'major_principle'] = None
        df.at[i,'is_major_principle_here'] = False
    else:
        df.at[i,'major_principle'] = row.idxmax()
        df.at[i,'is_major_principle_here'] = True
df.head()

,text,transparency,dignity,sustainability,trust,freedom and autonomy,beneficence,privacy,responsibility,non-maleficence,justice and equity,major_principle,is_major_principle_here
0,An intelligent system connected to the video s...,2.0,0.0,0.0,2.0,0.0,0.0,3.0,2.0,0.0,1.0,privacy,True
1,Terminal area and booking documents and bookin...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,transparency,True
2,Before a person with a vehicle enters the port...,5.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,0.0,1.0,transparency,True
3,A clear advertisement is placed on the booking...,5.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,transparency,True
4,Data information policy is communicated at the...,5.0,1.0,0.0,3.0,2.0,1.0,4.0,4.0,1.0,1.0,transparency,True


In [ ]:
df = df[['text', 'major_principle', 'is_major_principle_here']]
df['ethical_us'] = 'empty'
df = df.loc[df.is_major_principle_here, :]
df

<ipython-input-16-52086a981336>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ethical_us'] = 'empty'


,text,major_principle,is_major_principle_here,ethical_us
0,An intelligent system connected to the video s...,privacy,True,empty
1,Terminal area and booking documents and bookin...,transparency,True,empty
2,Before a person with a vehicle enters the port...,transparency,True,empty
3,A clear advertisement is placed on the booking...,transparency,True,empty
4,Data information policy is communicated at the...,transparency,True,empty
...,...,...,...,...
151,The system shall display and embed in each mes...,transparency,True,empty
152,All network transactions that involve financia...,trust,True,empty
153,System use shall not cause any harm to human u...,non-maleficence,True,empty
154,The system should warn ECA users about the mal...,non-maleficence,True,empty


In [ ]:
client = OpenAI()

for i, row in df.iterrows():
    messages = [
        {"role": "system", "content": "You are an ethical requirements engineer translating requirements to ethical user stories based on one of the 10 ethical principles:\n"},
        {"role": "system", "content": "Transparency: Transparency can typically be understood in two ways: the transparency of the AI technology itself and the transparency of the AI organisations developing and using it. Throughout our analysis, transparency was regularly discussed directly, or in relation to processes required to ensure it, such as explainability, understandability and communication.\n"},
        {"role": "system", "content": "Non-maleficence: The principle of non-maleficence gained attention, and in its most basic form, it means to do no harm or avoid doing harm to others\n"},
        {"role": "system", "content": "Responsibility: The principle of responsibility is base on responsibility of the developers and stakeholders over the AI, accountability, liability and acting with integrity\n"},
        {"role": "system", "content": "Privacy: Related to privacy of personal data, because of the large abundance of data that is required for AI to work, it is important that individuals privacy is not jeopardised as a result\n"},
        {"role": "system", "content": "Beneficence: Beneficence essentially means to do good, to carry out an activity with the intention of benefitting someone or society as a whole\n"},
        {"role": "system", "content": "Freedom and autonomy: Democratic societies place value in freedom and autonomy, and it is important that AI use does not encumber or harm these for us\n"},
        {"role": "system", "content": "Sustainability: All fields and disciplines are affected and need to incorporate sustainability agendas\n"},
        {"role": "system", "content": "Dignity: Human dignity is the recognition that individuals have inherent worth and that their rights should be respected\n"},
        {"role": "system", "content": "Justice and fairness: The issue of discrimination and unfair results resulting from algorithms has become a significant concern. It is imperative that systems are designed to ensure that they are free from any form of unfairness and inequality.\n"},
        {"role": "system", "content": "Trust: Trust is built by keeping promises, making sure systems work properly and protecting data responsibly. Organisations must prove their trustworthiness by ensuring that their technologies are secure and effective.\n"},
        {"role": "system", "content": "and the TEMPLATE:\n Title: <title>, Description: As a <persona> i want to <do something> <so that>, Work: <work>\n \n\n"},
        {"role": "user", "content": f"Now following this template and the ethical principles definition, transform the requirement {row['text']} into an ethical user story based on the principle {row['major_principle']} containing a title, substituting <title> for a descriptive user story title, description in one line of an ethical user story, substituting <persona> for the one that requires this functionality, <do something> for what the <persona> wants to do and <so that> to the end goal of the functionality and work, substituting <work> with what needs to be done so that the user story can be considered done"}
    ]


    response = client.chat.completions.create(
      model="gpt-4o",
      messages=messages
    )
    df.at[i,'ethical_us'] = response.choices[0].message.content


In [ ]:
print(df['ethical_us'][2])

Title: Vehicle Information Transparency Prior to Port Entry

Description: As a vehicle owner, I want to be clearly informed about what information will be handled before entering the port so that I can understand and consent to data usage during the reservation process.

Work: Develop a clear and detailed explanation process, presented at the reservation stage, covering how vehicle information will be used, stored, and managed, ensuring the information is easily understandable before the data is collected.


In [ ]:
df['correct'] = df['ethical_us'].str.contains('Title')
df['correct'].value_counts()
df

,text,major_principle,is_major_principle_here,ethical_us,correct
0,An intelligent system connected to the video s...,privacy,True,Title: Privacy-Respecting Intelligent Surveill...,True
1,Terminal area and booking documents and bookin...,transparency,True,Title: Transparent Help Information for Termin...,True
2,Before a person with a vehicle enters the port...,transparency,True,Title: Vehicle Information Transparency Prior ...,True
3,A clear advertisement is placed on the booking...,transparency,True,Title: Transparent Communication of AI and Dat...,True
4,Data information policy is communicated at the...,transparency,True,Title: Transparent Data Handling and GDPR Comp...,True
...,...,...,...,...,...
151,The system shall display and embed in each mes...,transparency,True,Title: Ensure Message Transparency with Timest...,True
152,All network transactions that involve financia...,trust,True,Title: Secure Encryption of Sensitive Transact...,True
153,System use shall not cause any harm to human u...,non-maleficence,True,Title: Ensure Harm-Free System Usage\n\nDescri...,True
154,The system should warn ECA users about the mal...,non-maleficence,True,Title: Malfunction Warning Alert for ECA Users...,True


In [ ]:
df.to_csv('./requirements-to-us/data/dataset.csv')